In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import re
import math

class CReLU(nn.Module):
    def __init__(self):
        super(CReLU, self).__init__()
    def forward(self, x):
        return torch.cat((F.relu(x), F.relu(-x)), 0)

class Anime4kRestore(nn.Module):
    def __init__(self):
        super(Anime4kRestore, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=4, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=4, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=8, out_channels=4, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(in_channels=8, out_channels=3, kernel_size=3, padding=1)
        self.crelu = CReLU()
    
    def forward(self, x):
        # t = self.crelu(torch.clamp(self.conv1(x), max= 1.0))
        # t = self.crelu(torch.clamp(self.conv2(t), max= 1.0))
        # t = self.crelu(torch.clamp(self.conv3(t), max= 1.0))
        t = self.crelu(self.conv1(x))
        t = self.crelu(self.conv2(t))
        t = self.crelu(self.conv3(t))
        return torch.clamp(self.conv4(t) + x, max= 1.0, min=0.0)

    
def convert(c, iter):
    out_chan, in_chan, width, height = c.weight.shape
    # print(c.weight.shape)
    for t in range(math.ceil(in_chan/4)):
        for w in range(width):
            for h in range(height):
                for i in range(min(4,in_chan)):
                    for o in range(out_chan):
                        c.weight.data[o,t*4+i,w,h] = float(next(iter).group(0))
    for i in range(len(c.bias)):
        c.bias.data[i] = float(next(iter).group(0))

model = Anime4kRestore()#.half()
for param in model.parameters():
    param.requires_grad = False

if False:
    dummy_input = torch.randn(12, 3, 224, 224)#.half()
    torch.onnx.export(model, dummy_input, "simple_model.onnx")   

with open("tmp/Anime4K_Restore_CNN_S.glsl") as f:
    text = f.read()
    
pattern = r'-?\d+(\.\d{2,})(e-?\d+)?'
re_iter = re.finditer(pattern,text)

convert(model.conv1, re_iter)
convert(model.conv2, re_iter)
convert(model.conv3, re_iter)
convert(model.conv4, re_iter)

# print(model.conv1.weight[0,0,0,:])
# print(model.conv1.weight[0,0,:,0])
# print(model.conv1.weight[0,:,0,0])
# print(model.conv1.weight[:,0,0,0])
# print("-------------------------------")
# print(model.conv4.weight[0,0,0,:])
# print(model.conv4.weight[0,0,:,0])
# print(model.conv4.weight[0,:,0,0])
# print(model.conv4.weight[:,0,0,0])

#---------------------------------------------------------------------------------------------

import torchvision
from IPython.display import Image, display
from PIL import Image
to_pil = torchvision.transforms.ToPILImage()
to_tensor = torchvision.transforms.ToTensor()

image = torchvision.io.read_image("/Users/khoi.ho/Downloads/110834364_p0.jpg",torchvision.io.ImageReadMode.RGB) / 256

image2 = Image.open("/Users/khoi.ho/Downloads/Screenshot 2023-08-21 14-04-25.png").convert("RGB")
image2 = to_tensor(image2)

# image2[[1, 2], :, :] = image2[[2, 1], :, :]
# image2[[1, 0], :, :] = image2[[0, 1], :, :]

# display(to_pil(image2))

# out = (model(image2) / 2 + 0.5)
out = model(image2)
print(torch.min(out))
print(torch.max(out))
min_out = torch.min(out)
max_out = torch.max(out)
out = (out - min_out)/(max_out-min_out)


display(to_pil(out))


In [ ]:
out = model(image2)
display(to_pil(out))

In [50]:
from functools import reduce
import re
mul = lambda l: reduce(lambda x, y: x*y, l)

c = nn.Conv2d(in_channels=3, out_channels=4, kernel_size=3, padding=1)
c = nn.Conv2d(in_channels=8, out_channels=3, kernel_size=3, padding=1)

# c = nn.Conv2d(in_channels=4, out_channels=4, kernel_size=1, padding="same")
text = '''
#define go_0(x_off, y_off) (MAIN_texOff(vec2(x_off, y_off)))
vec4 hook() {
    vec4 result = mat4(-0.19288683, -0.21397883, 0.111997396, -0.04791413, -0.26682988, -0.06144587, -0.03601853, -0.16693151, 0.038494494, -0.16651472, 0.147657, -0.083003886, 0.0, 0.0, 0.0, 0.0) * go_0(-1.0, -1.0);
    result += mat4(-0.14286195, 0.08746566, -0.40107322, 0.12390977, -0.33392772, -0.18703035, -0.21326795, 0.04780781, -0.15155545, -0.0010025925, -0.1554875, -0.10676251, 0.0, 0.0, 0.0, 0.0) * go_0(-1.0, 0.0);
    result += mat4(0.28095165, 0.022872915, -0.21342312, -0.29982176, 0.025937587, -0.055012174, -0.33779636, 0.0015666655, 0.076416336, 0.06656033, -0.1557806, 0.1078894, 0.0, 0.0, 0.0, 0.0) * go_0(-1.0, 1.0);
    result += mat4(-0.31584853, 0.07527119, 0.30713862, -0.34014285, -0.50103146, -0.07217874, 0.512807, -0.09597398, -0.32097813, -0.051580857, -0.022466356, 0.01148551, 0.0, 0.0, 0.0, 0.0) * go_0(0.0, -1.0);
    result += mat4(-0.026032459, -0.04193211, 0.37703893, -0.031916667, -0.27421117, 1.0906446, -0.049654085, -0.19814016, 0.07819544, 0.06003738, 0.1405805, -0.0064135445, 0.0, 0.0, 0.0, 0.0) * go_0(0.0, 0.0);
    result += mat4(0.041450135, 0.11319654, -0.23237701, 0.08443178, 0.53344345, 0.30857387, -0.057264958, -0.1575803, 0.2325609, -0.027797326, -0.04544767, -0.18720597, 0.0, 0.0, 0.0, 0.0) * go_0(0.0, 1.0);
    result += mat4(0.2531829, -0.074966915, -0.27800754, -0.3146097, 0.20126024, -0.5380133, -0.15082566, -0.19021043, 0.29951036, 0.17123336, -0.01681872, -0.12574998, 0.0, 0.0, 0.0, 0.0) * go_0(1.0, -1.0);
    result += mat4(0.25203633, 0.19882993, 0.14906439, 0.13593598, 0.40712556, 0.084902965, 0.42969635, 0.2961132, -0.057267334, -0.030388135, 8.8084314e-05, 0.0210724, 0.0, 0.0, 0.0, 0.0) * go_0(1.0, 0.0);
    result += mat4(-0.13459359, -0.12199573, 0.12591946, 0.24736497, 0.2033463, -0.09388599, -0.094370656, 0.1071285, -0.18479438, -0.066625565, 0.08279283, 0.20130983, 0.0, 0.0, 0.0, 0.0) * go_0(1.0, 1.0);
    result += vec4(-0.011108127, -0.07481861, 0.07640154, 0.4964964);
    return result;
}'''

text2 = '''
#define go_0(x_off, y_off) (max((conv2d_2_tf_texOff(vec2(x_off, y_off))), 0.0))
#define go_1(x_off, y_off) (max(-(conv2d_2_tf_texOff(vec2(x_off, y_off))), 0.0))
vec4 hook() {
    vec4 result = mat4(0.15873, 0.17989138, 0.14648493, 0.0, -0.017379675, -0.017363746, -0.019855022, 0.0, 0.009670625, 0.0070157526, 0.0075994316, 0.0, 0.025388412, 0.027231036, 0.024052646, 0.0) * go_0(-1.0, -1.0);
    result += mat4(0.048195973, 0.041760173, 0.037366055, 0.0, -0.115950756, -0.12887983, -0.12535639, 0.0, 0.032125086, 0.03397254, 0.032950625, 0.0, 0.01223746, 0.020822672, 0.0161561, 0.0) * go_0(-1.0, 0.0);
    result += mat4(0.0890567, 0.094453335, 0.09014035, 0.0, 0.016081346, 0.017434116, 0.020783134, 0.0, -0.011775135, -0.010094134, -0.018522855, 0.0, 0.072103254, 0.07940666, 0.065876864, 0.0) * go_0(-1.0, 1.0);
    result += mat4(-0.04841196, -0.06963968, -0.056574684, 0.0, 0.10912542, 0.11813441, 0.10643838, 0.0, -0.013013885, -0.01562045, -0.013802797, 0.0, 0.037505716, 0.04352026, 0.04645123, 0.0) * go_0(0.0, -1.0);
    result += mat4(-0.3472869, -0.36243078, -0.33530185, 0.0, 0.23654196, 0.2305048, 0.22150646, 0.0, -0.045226905, -0.041799217, -0.042511635, 0.0, -0.10267792, -0.1123385, -0.10845448, 0.0) * go_0(0.0, 0.0);
    result += mat4(0.011987401, 0.012285043, 0.007813165, 0.0, -0.15911353, -0.17523928, -0.1535267, 0.0, 0.15675929, 0.16531634, 0.15948962, 0.0, -0.09240023, -0.09513292, -0.084187366, 0.0) * go_0(0.0, 1.0);
    result += mat4(0.069052905, 0.07278333, 0.0756627, 0.0, -0.012180326, -0.018794727, -0.031050753, 0.0, -0.044663202, -0.04362803, -0.038904265, 0.0, -0.008540197, -0.011201734, -0.01556625, 0.0) * go_0(1.0, -1.0);
    result += mat4(-0.08261173, -0.09042543, -0.07589266, 0.0, 0.043515377, 0.045066774, 0.04037769, 0.0, -0.06262993, -0.07469342, -0.058593787, 0.0, 0.026696987, 0.028740842, 0.037405368, 0.0) * go_0(1.0, 0.0);
    result += mat4(0.07975598, 0.09597654, 0.08997132, 0.0, -0.07844719, -0.07880916, -0.06835411, 0.0, 0.05668995, 0.050163813, 0.053357534, 0.0, -0.020040333, -0.019867316, -0.01907621, 0.0) * go_0(1.0, 1.0);
    
    result += mat4(-0.017078733, -0.017393313, -0.008266595, 0.0, -0.0033478448, -0.0027439648, -0.0042334674, 0.0, -0.06354017, -0.062058125, -0.04652064, 0.0, -0.010787706, -0.0062706997, -0.007573461, 0.0) * go_1(-1.0, -1.0);
    result += mat4(-0.019895451, -0.016341688, -0.008712399, 0.0, 0.026231976, 0.023955572, 0.0216376, 0.0, -0.061950512, -0.05481285, -0.05261985, 0.0, -0.018804235, -0.016235247, -0.0131616965, 0.0) * go_1(-1.0, 0.0);
    result += mat4(-0.055628926, -0.063315354, -0.057192408, 0.0, -0.0256364, -0.028660972, -0.02937357, 0.0, -0.017604912, -0.020851422, -0.016070362, 0.0, -0.0870202, -0.0832279, -0.07525406, 0.0) * go_1(-1.0, 1.0);
    result += mat4(0.062738225, 0.07106593, 0.061644047, 0.0, -0.06068257, -0.06983662, -0.066070385, 0.0, 0.024919355, 0.03227179, 0.028569462, 0.0, -0.07866227, -0.098967604, -0.092128105, 0.0) * go_1(0.0, -1.0);
    result += mat4(0.040397774, 0.047241107, 0.03962998, 0.0, -0.09112752, -0.10057507, -0.09301817, 0.0, 0.10833967, 0.101835825, 0.10027467, 0.0, 0.27189335, 0.27433604, 0.26781923, 0.0) * go_1(0.0, 0.0);
    result += mat4(-0.044211388, -0.042373534, -0.03658007, 0.0, 0.113148406, 0.12423258, 0.107804194, 0.0, -0.17081551, -0.18562958, -0.17475435, 0.0, 0.09636739, 0.10763415, 0.093332425, 0.0) * go_1(0.0, 1.0);
    result += mat4(-0.03798545, -0.047811143, -0.050768293, 0.0, 0.018775463, 0.026812987, 0.03452908, 0.0, 0.0055677597, 0.0039081173, -0.0017878668, 0.0, -0.10728597, -0.12618187, -0.109045394, 0.0) * go_1(1.0, -1.0);
    result += mat4(0.06359783, 0.064184755, 0.04934199, 0.0, -0.009819327, -0.006616115, -0.007431496, 0.0, 0.025055679, 0.024787048, 0.017360551, 0.0, -0.047140837, -0.061695747, -0.06440822, 0.0) * go_1(1.0, 0.0);
    result += mat4(0.060199022, 0.06482763, 0.059514645, 0.0, 0.026998974, 0.028776823, 0.024897143, 0.0, 0.17968474, 0.19337215, 0.16760105, 0.0, 0.0075838566, 0.010503482, 0.011993149, 0.0) * go_1(1.0, 1.0);
    result += vec4(-0.0052927984, -0.0060193934, -0.0048643993, 0.0);
    return result + MAIN_tex(MAIN_pos);
}'''

print(mul(c.weight.shape)+len(c.bias))
pattern = r'-?\d+(\.\d{2,})(e-?\d+)?'
re_iter = re.finditer(pattern,text2)
print(c.weight.shape)
in_chan, out_chan, width, height = c.weight.shape
for t in range(2):
    for w in range(width):
        for h in range(height):
            for o in range(4):
                for i in range(in_chan):
                    c.weight.data[i,4*t+o,w,h] = float(next(re_iter).group(0))
for i in range(len(c.bias)):
    c.bias.data[i] = float(next(re_iter).group(0))

d = torch.ones(8, 5, 5)
print(c.weight[0,0,0,:])
print(c.weight[0,0,:,0])
print(c.weight[0,:,0,0])
print(c.weight[:,0,0,0])

219
torch.Size([3, 8, 3, 3])
tensor([0.1587, 0.0482, 0.0891], grad_fn=<SliceBackward0>)
tensor([ 0.1587, -0.0484,  0.0691], grad_fn=<SelectBackward0>)
tensor([ 0.1587, -0.0174,  0.0097,  0.0254, -0.0171, -0.0033, -0.0635, -0.0108],
       grad_fn=<SelectBackward0>)
tensor([0.1587, 0.1799, 0.1465], grad_fn=<SelectBackward0>)
